In [ ]:
our_language = 'pt'

In [ ]:
# https://github.com/korakot/kora/blob/416481ca3ef67b20ba4380c99df6d04651b170f0/kora/record.py#L7

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
        blob = new Blob(chunks)
        text = await b2text(blob)
        resolve(text)
    }
    recorder.stop()
})
"""

def record(sec=5):
    display(Javascript(RECORD))
    js_result = output.eval_js('record(%s)' % (sec * 1000)) #passando em milisengundos
    audio = b64decode(js_result.split(',')[1])
    file_name = 'request_audio.wav'
    with open(file_name,'wb') as f:
      f.write(audio)
    return f'/content/{file_name}'

print("Já pode falar\n")
record_file = record()
display(Audio(record_file, autoplay = True))

Já pode falar



<IPython.core.display.Javascript object>

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

In [ ]:
import whisper

model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language = our_language)
transcription = result["text"]
print(transcription)

 Quanto anos você tem já tchati PT?


In [ ]:
!pip install openai

In [ ]:
import openai

openai.api_key = "YOUR_KEY_OPENAI"

# Envia uma requisição à API do ChatCompletion usando o modelo GPT-3.5 Turbo
# Lembrando que, a variável 'transcription' contém a transcrição do nosso áudio.
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": transcription,
        },
    ],
)
# Obtém a resposta gerada pelo ChatGPT
chatgpt_response = response.choices[0].message.content
print(chatgpt_response)

Eu sou uma inteligência artificial, então não tenho uma idade física. Fui lançada em 2020, então estou disponível para conversar e ajudar desde então.


In [ ]:
!pip install gTTS

In [ ]:
from gtts import gTTS

gtts_object = gTTS(text=chatgpt_response, lang=our_language, slow=False)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)
display(Audio(response_audio, autoplay = True))